# Python 异步编程

本教程将学习Python中的异步编程，包括asyncio模块的使用，帮助你编写高效的异步程序。

## 1. 异步编程基础 - asyncio


In [2]:
import asyncio
import time

# 定义一个简单的异步函数
async def say_hello(name, delay):
    """异步函数示例"""
    print(f"开始: {name}")
    await asyncio.sleep(delay)  # 模拟异步操作
    print(f"完成: {name}")
    return f"Hello from {name}"

# 运行异步函数
async def main():
    """主异步函数"""
    print("开始异步执行...")
    
    # 顺序执行（慢）
    start = time.time()
    await say_hello("任务1", 1)
    await say_hello("任务2", 1)
    end = time.time()
    print(f"顺序执行耗时: {end - start:.2f} 秒\n")
    
    # 并发执行（快）
    start = time.time()
    await asyncio.gather(
        say_hello("任务A", 1),
        say_hello("任务B", 1),
        say_hello("任务C", 1)
    )
    end = time.time()
    print(f"并发执行耗时: {end - start:.2f} 秒")

# 运行异步程序
await main()


开始异步执行...
开始: 任务1
完成: 任务1
开始: 任务2
完成: 任务2
顺序执行耗时: 2.01 秒

开始: 任务A
开始: 任务B
开始: 任务C
完成: 任务A
完成: 任务B
完成: 任务C
并发执行耗时: 1.01 秒


## 2. 异步任务和Future


In [3]:
async def fetch_data(url, delay):
    """模拟异步获取数据"""
    print(f"开始获取: {url}")
    await asyncio.sleep(delay)  # 模拟网络延迟
    print(f"完成获取: {url}")
    return f"数据来自 {url}"

async def main():
    # 创建任务
    task1 = asyncio.create_task(fetch_data("url1", 1))
    task2 = asyncio.create_task(fetch_data("url2", 2))
    task3 = asyncio.create_task(fetch_data("url3", 1))
    
    # 等待所有任务完成
    results = await asyncio.gather(task1, task2, task3)
    
    print("\n所有结果:")
    for result in results:
        print(f"  - {result}")

await main()


开始获取: url1
开始获取: url2
开始获取: url3
完成获取: url1
完成获取: url3
完成获取: url2

所有结果:
  - 数据来自 url1
  - 数据来自 url2
  - 数据来自 url3


## 3. 异步上下文管理器


In [4]:
class AsyncFileManager:
    """异步文件管理器"""
    
    async def __aenter__(self):
        """进入异步上下文"""
        print("打开文件（异步）")
        await asyncio.sleep(0.1)  # 模拟异步操作
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """退出异步上下文"""
        print("关闭文件（异步）")
        await asyncio.sleep(0.1)
        return False
    
    async def write(self, data):
        """异步写入"""
        print(f"写入数据: {data}")
        await asyncio.sleep(0.1)

async def main():
    async with AsyncFileManager() as file:
        await file.write("Hello, Async World!")

await main()


打开文件（异步）
写入数据: Hello, Async World!
关闭文件（异步）


## 4. 异步迭代器


In [5]:
class AsyncCounter:
    """异步计数器（异步迭代器）"""
    
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.current = start
    
    def __aiter__(self):
        """返回异步迭代器"""
        return self
    
    async def __anext__(self):
        """返回下一个值"""
        if self.current >= self.end:
            raise StopAsyncIteration
        
        await asyncio.sleep(0.1)  # 模拟异步操作
        value = self.current
        self.current += 1
        return value

async def main():
    print("异步迭代器示例:")
    async for value in AsyncCounter(1, 6):
        print(f"值: {value}")

await main()


异步迭代器示例:
值: 1
值: 2
值: 3
值: 4
值: 5


## 5. 异步锁和信号量


In [6]:
# 异步锁
async def worker_with_lock(name, lock, shared_resource):
    """带锁的工作函数"""
    async with lock:  # 获取锁
        print(f"{name} 获得锁")
        shared_resource['counter'] += 1
        await asyncio.sleep(0.1)  # 模拟工作
        print(f"{name} 释放锁，计数器: {shared_resource['counter']}")

async def main():
    lock = asyncio.Lock()
    shared_resource = {'counter': 0}
    
    # 创建多个任务，使用锁保护共享资源
    tasks = [
        worker_with_lock(f"任务{i}", lock, shared_resource)
        for i in range(5)
    ]
    
    await asyncio.gather(*tasks)
    print(f"\n最终计数器值: {shared_resource['counter']}")

await main()


任务0 获得锁
任务0 释放锁，计数器: 1
任务1 获得锁
任务1 释放锁，计数器: 2
任务2 获得锁
任务2 释放锁，计数器: 3
任务3 获得锁
任务3 释放锁，计数器: 4
任务4 获得锁
任务4 释放锁，计数器: 5

最终计数器值: 5


In [7]:
# 异步信号量（限制并发数）
async def limited_task(name, semaphore):
    """受信号量限制的任务"""
    async with semaphore:
        print(f"{name} 开始执行")
        await asyncio.sleep(1)  # 模拟工作
        print(f"{name} 完成")

async def main():
    semaphore = asyncio.Semaphore(2)  # 最多同时执行2个任务
    
    tasks = [
        limited_task(f"任务{i}", semaphore)
        for i in range(5)
    ]
    
    print("使用信号量限制并发数（最多2个）:")
    await asyncio.gather(*tasks)

await main()


使用信号量限制并发数（最多2个）:
任务0 开始执行
任务1 开始执行
任务0 完成
任务1 完成
任务2 开始执行
任务3 开始执行
任务2 完成
任务3 完成
任务4 开始执行
任务4 完成
